# Three ways to connect to milvus from a notebook

There are 5 different ways to start up a Milvus server.
1.  [Milvus lite](https://milvus.io/docs/milvus_lite.md) runs a local server.  ⛔️ Milvus lite is only meant for demos and local testing.

💡 **For production purposes**, it is recommended to use Milvus local docker, clusters, or fully-managed Milvus on Zilliz Cloud. <br>

2. [Milvus local docker](https://milvus.io/docs/install_standalone-docker.md) requires local docker installed and running.
3. [Ziliz Cloud free tier](#zilliz_free)
4. <a href="https://milvus.io/docs/install_cluster-milvusoperator.md">Milvus clusters</a> requires a K8s cluster up and running.
5. <a href="https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.milvus.Milvus.html">LangChain Milvus</a> and other 3rd party adapters.

Once your milvus server or cluster is up and running, you're all set to make API calls.  I'll demonstrate how to do this using the [Python SDK](https://github.com/milvus-io/pymilvus/blob/master/pymilvus/milvus_client/milvus_client.py).  You can also check out a [practical example](https://github.com/milvus-io/pymilvus/blob/bac31951d5c5a9dacb6632e535e3c4d284726390/examples/hello_milvus_simple.py).  

### Notebook contents for 3 ways to connect
1. [Milvus lite demo](#milvus_lite)
2. [Local milvus on docker demo](#milvus_local)
3. [Zilliz cloud free tier](#zilliz_free)
4. Milvus clusters on K8 - demo coming soon
4. 3rd party adapters - demo coming soon

In [1]:
# Import common libraries.
import time

# Import milvus
# !pip install milvus pymilvus
import milvus, pymilvus #pymilvus is the python sdk

milvus_version = milvus.__version__
pymilvus_version = pymilvus.__version__
print(f"milvus: {milvus_version}")
print(f"pymilvus: {pymilvus_version}")

# Check major + minor versions match.
assert pymilvus_version.split(sep=".")[0]==milvus_version.split(sep=".")[0]
assert pymilvus_version.split(sep=".")[1]==milvus_version.split(sep=".")[1]

milvus: 2.3.0
pymilvus: 2.3.0


In [2]:
# Props to Sebastian Raschka for this handy watermark.
# !pip install watermark

%load_ext watermark
%watermark -a 'Christy Bergman' -v -p torch,transformers,milvus,pymilvus,langchain --conda

Author: Christy Bergman

Python implementation: CPython
Python version       : 3.10.12
IPython version      : 8.15.0

torch       : 2.0.1
transformers: 4.33.2
milvus      : 2.3.0
pymilvus    : 2.3.0
langchain   : 0.0.301

conda environment: py310



## 1. Milvus lite <a class="anchor" id="milvus_lite"></a>
This section uses [Milvus lite](https://milvus.io/docs/milvus_lite.md), which runs a local server.
- pip install milvus pymilvus

In [3]:
from milvus import default_server, debug_server
from pymilvus import connections, utility

# (OPTIONAL) Set if you want store all related data to specific location
# Default location:
#   %APPDATA%/milvus-io/milvus-server on windows
#   ~/.milvus-io/milvus-server on linux
# default_server.set_base_dir('milvus_data')

# Cleanup previous data and stop server in case it is still running.
default_server.stop()
default_server.cleanup()

# Start a new milvus-lite local server.
start_time = time.time()
default_server.start()
end_time = time.time()

print(f"startup time: {end_time - start_time}")
# startup time: 5.6739208698272705

startup time: 8.414561033248901


In [4]:
# Print port number
port = default_server.listen_port
print(port)

19530


In [5]:
# Add wait to avoid error message from trying to connect.
time.sleep(1)

# Now you could connect with localhost and the given port
# Port is defined by default_server.listen_port
connections.connect(host='127.0.0.1', 
                  port=default_server.listen_port,
                  show_startup_banner=True)

# Get server version.
print(utility.get_server_version())

v2.2-testing-20230824-68-ga34a9d606-lite


In [6]:
# Shut down and cleanup the milvus server.
default_server.stop()
default_server.cleanup()

## 2. Milvus local server running on Docker <a class="anchor" id="milvus_local"></a>

This section uses [Milvus local](https://milvus.io/docs/configure-docker.md) on Docker on your laptop. <br>
>⛔️ Make sure you pip install the correct version of milvus, pymilvus, and server yml file.  **Versions (major and minor) should all match**.

1. [Install Docker](https://docs.docker.com/get-docker/)
2. Start your Docker Desktop
3. Download the latest [docker-compose.yml](https://milvus.io/docs/install_standalone-docker.md#Download-the-YAML-file) (or run the wget command in the next cell)
4. From you terminal:  
   - activate your python environment
   - cd into directory where you saved the .yml file (usualy same dir as this notebook)
   - docker compose up -d
   - verify (either in terminal or on Docker Desktop) the containers are running
5. Import milvus
6. Connect to the local milvus server (see notebook code below)

In [7]:
# Download the latest .yaml file
# !wget https://github.com/milvus-io/milvus/releases/download/v2.3.0/milvus-standalone-docker-compose.yml -O docker-compose.yml

# Stop local milvus.
!docker compose down

# Start Docker - see terminal directions above.
!docker compose up -d

[+] Running 4/0
 ✔ Network milvus               Created                                    0.0s 
 ✔ Container milvus-minio       Created                                    0.0s 
 ✔ Container milvus-etcd        Created                                    0.0s 
 ✔ Container milvus-standalone  Created                                    0.0s 
[+] Running 4/4
 ✔ Network milvus               Created                                    0.0s 
 ✔ Container milvus-minio       Created                                    0.0s 
 ✔ Container milvus-etcd        Created                                    0.0s 
 ✔ Container milvus-standalone  Created                                    0.0s 
[+] Running 4/4
 ✔ Network milvus               Created                                    0.0s 
 ✔ Container milvus-minio       Created                                    0.0s 
 ✔ Container milvus-etcd        Created                                    0.0s 
 ✔ Container milvus-standalone  Created                      

In [8]:
# Import milvus
from pymilvus import (
    connections,
    utility,
    FieldSchema, CollectionSchema, DataType,
    Collection,
)

# Check if the containers are up and running.
!docker compose ps

NAME                IMAGE                                      COMMAND                                                                                                        SERVICE      CREATED        STATUS                                     PORTS
milvus-etcd         quay.io/coreos/etcd:v3.5.5                 "etcd -advertise-client-urls=http://127.0.0.1:2379 -listen-client-urls http://0.0.0.0:2379 --data-dir /etcd"   etcd         1 second ago   Up Less than a second (health: starting)   2379-2380/tcp
milvus-minio        minio/minio:RELEASE.2023-03-20T20-16-18Z   "/usr/bin/docker-entrypoint.sh minio server /minio_data --console-address :9001"                               minio        1 second ago   Up Less than a second (health: starting)   0.0.0.0:9000-9001->9000-9001/tcp
milvus-standalone   milvusdb/milvus:v2.3.0                     "/tini -- milvus run standalone"                                                                               standalone   1 second ago   Up Less th

In [9]:
# Verify which local port the Milvus server is listening on
!docker port milvus-standalone #19530/tcp

9091/tcp -> 0.0.0.0:9091
19530/tcp -> 0.0.0.0:19530


In [10]:
# Add wait to avoid error message from trying to connect.
time.sleep(20)

# Connect to the local server.
connections.connect("local", host="0.0.0.0", port="19530")
print("Connected to Milvus.")

# Get server version.
print(utility.get_server_version())

Connected to Milvus.
v2.3.0-dev


**Error message if try to connect before docker up.**

MilvusException                           Traceback (most recent call last)
/Users/christybergman/Documents/christy_github/ZillizDemos/milvus_onboarding/milvus_connect.ipynb Cell 13 line 5
      2 time.sleep(10)
      4 # Connect to the local server.
----> 5 connections.connect("local", host="0.0.0.0", port="19530")
...
    137     ) from e
    138 except Exception as e:
    139     raise e from e

MilvusException: <MilvusException: (code=2, message=Fail connecting to server on 0.0.0.0:19530. Timeout)>

In [11]:
# Check if a collection already exists.
collection_name = "movies"
has = utility.has_collection(collection_name)
print(f"Collection '{collection_name}' exists in Milvus?")
print(f"Answer --> {has}!")
if has:
    drop_result = utility.drop_collection(collection_name)
    print(f"Successfully dropped {collection_name}")

Collection 'movies' exists in Milvus?
Answer --> False!


In [12]:
# Stop local milvus.
!docker compose down

[+] Running 0/0
 ⠋ Container milvus-standalone  Stopping                                   0.1s 
[+] Running 1/1
 ✔ Container milvus-standalone  Removed                                    0.1s 
 ⠋ Container milvus-etcd        Stopping                                   0.1s 
 ⠋ Container milvus-minio       Stopping                                   0.1s 
[+] Running 2/3
 ✔ Container milvus-standalone  Removed                                    0.1s 
 ✔ Container milvus-etcd        Removed                                    0.1s 
 ⠙ Container milvus-minio       Stopping                                   0.2s 
[+] Running 2/3
 ✔ Container milvus-standalone  Removed                                    0.1s 
 ✔ Container milvus-etcd        Removed                                    0.1s 
 ⠹ Container milvus-minio       Stopping                                   0.3s 
[+] Running 2/3
 ✔ Container milvus-standalone  Removed                                    0.1s 
 ✔ Container milvus-etcd     

## 3. Zilliz free tier  <a class="anchor" id="zilliz_free"></a>

This section uses [Zilliz](https://zilliz.com), free tier.  If you have not already, sign up for a [free trial](https://cloud.zilliz.com/signup).  

If you already have a Zilliz account and want to use free tier, just be sure to select "Starter" option when you [create your cluster](https://docs.zilliz.com/docs/create-cluster).  ❤️‍🔥 **In other words, everybody gets free tier!!**  
- One free tier cluster per account.
- Per free tier cluster, up to two collections at a time. (Think of a collection like a database table. Each collection has an index, schema, and consistency-level).
- Each free tier collection can support up to 1M vectors (Think of this like rows in a database table).

If you require additional clusters, we recommend subscribing to either our Standard or Enterprise plan.

### 👩 Set up instructions for Zilliz 

1. From [cloud.zilliz.com](cloud.zilliz.com), click **"+ Create Cluster"**
2. Select <i>**Starter**</i> option for the cluster and click **"Next: Create Collection"**
   <div>
   <img src="../images/zilliz_cluster_choose.png" width="60%"/>
   </div>

1. Name your collection with a <i>**Collection Name**</i> and click **"Create Collection and Cluster"**.
2. From the Clusters page, 
   - copy the cluster uri and save somewhere locally.
   - copy your cluster API KEY.  Keep this private! 
     <div>
     <img src="../images/zilliz_cluster_uri_token.png" width="80%"/>
     </div>

3. Add the API KEY to your environment variables.  See this [article for instructions](https://help.openai.com/en/articles/5112595-best-practices-for-api-key-safety) how in either Windows or Mac/Linux environment.
4. In Jupyter, you'll also need .env file (in same dir as notebooks) containing lines like this:
   - ZILLIZ_API_KEY=value
5. In your code, connect to your Zilliz cluster, see code example below.

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
TOKEN = os.getenv("ZILLIZ_API_KEY")

# Connect to Zilliz cloud using enpoint URI and API key TOKEN.
CLUSTER_ENDPOINT="https://in03-xxxx.api.gcp-us-west1.zillizcloud.com:443"
connections.connect(
  alias='default',
  #  Public endpoint obtained from Zilliz Cloud
  uri=CLUSTER_ENDPOINT,
  # API key or a colon-separated cluster username and password
  token=TOKEN,
)

# Check if the server is ready and get colleciton name.
print(f"Type of server: {utility.get_server_version()}")